In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import numpy as np

# โหลดตัวแปรจาก .env
load_dotenv()

# ดึงค่าจาก environment
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

# สร้าง engine สำหรับเชื่อมต่อฐานข้อมูล
engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

# SQL query
query = """
SELECT quo_num,type_insure,type_work,datestart,id_government_officer,status_gpf , quo_num_old, type_status , type_key , app_type, chanel_key,status as status_fssp
FROM fin_system_select_plan 
WHERE datestart >= '2025-05-01' AND datestart < '2025-07-01'
AND type_insure IN ('ประกันรถ', 'ตรอ')
"""

df = pd.read_sql(query, engine)
df


In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv() 

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance_task'

engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

query = """
SELECT quo_num,order_number,worksend, chanel, datekey,status as status_fo
FROM fin_order

"""

df1 = pd.read_sql(query, engine)
df1


In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv() 

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

query = """
SELECT quo_num,datestart, numpay, show_price_ins, show_price_prb, show_price_total, show_price_check, show_price_service, show_price_taxcar, show_price_fine, show_price_addon, show_price_payment, distax, show_ems_price, show_discount_ins, discount_mkt, discount_government, discount_government_fin, discount_government_ins, coupon_addon,status as status_fsp
FROM fin_system_pay 
WHERE datestart >= '2025-05-01' AND datestart < '2025-07-01'
AND type_insure IN ('ประกันรถ', 'ตรอ')

"""

df2 = pd.read_sql(query, engine)
df2


In [ ]:
df_merged = pd.merge(df, df1, on='quo_num', how='left')
df_merged

In [ ]:
df_merged1 = pd.merge(df_merged, df2, on='quo_num', how='left')
df_merged1

In [ ]:
import pandas as pd

def fill_chanel_key(row):
    chanel_key = row['chanel_key']
    type_key = row['type_key']
    app_type = row['app_type']
    type_insure = row['type_insure']

    # ถ้า chanel_key มีค่าอยู่แล้ว
    if pd.notnull(chanel_key) and str(chanel_key).strip() != "":
        return chanel_key

    # ถ้าทั้ง type_key และ app_type ไม่ null
    if pd.notnull(type_key) and pd.notnull(app_type):
        if type_key == app_type:
            if type_insure == 'ตรอ':
                return f"{type_key} VIF"
            else:
                return type_key
        else:
            if type_key in app_type:
                base = app_type.replace(type_key, "").replace("-", "").strip()
                return f"{type_key} {base}" if base else type_key
            elif app_type in type_key:
                base = type_key.replace(app_type, "").replace("-", "").strip()
                return f"{app_type} {base}" if base else app_type
            else:
                return f"{type_key} {app_type}"

    # ถ้ามีแค่ type_key
    if pd.notnull(type_key) and (pd.isnull(app_type) or str(app_type).strip() == ""):
        if pd.notnull(type_insure) and str(type_insure).strip() != "":
            return f"{type_key} {type_insure}"
        else:
            return type_key

    # ถ้ามีแค่ app_type
    if pd.notnull(app_type) and (pd.isnull(type_key) or str(type_key).strip() == ""):
        if pd.notnull(type_insure) and str(type_insure).strip() != "":
            return f"{app_type} {type_insure}"
        else:
            return app_type

    # ไม่มีอะไรเลย
    return None

# apply กลับ
df_merged1['chanel_key'] = df_merged1.apply(fill_chanel_key, axis=1)
df_merged1

In [ ]:
df_merged1['chanel_key'] = df_merged1['chanel_key'].replace({
    'B2B': 'APP B2B',
    'WEB ตรอ': 'WEB VIF',
    'TELE': 'APP TELE',
    'APP-B2C': 'APP B2C',
    'APP ประกันรถ' : 'APP B2B',
    'WEB ประกันรถ': 'WEB'
})
df_merged1

In [ ]:
df_merged1.drop(columns=['type_key', 'app_type'], inplace=True)
df_merged1

In [ ]:
df_merged1.rename(columns={
    "quo_num": "quotation_num",
    "datestart_x": "quotation_date",
    "datestart_y": "transaction_date",
    "datekey": "order_time",
    "type_insure": "type_insurance",
    "type_work": "order_type",
    "id_government_officer": "rights_government",
    "status_gpf": "goverment_type",
    "type_status": "check_type",
    "type_key": "key_channel",
    "quo_num_old": "quotation_num_old",
    "numpay": "installment_number",
    "show_price_ins": "ins_amount",
    "show_price_prb": "prb_amount",
    "show_price_total": "total_amount",
    "show_price_check": "show_price_check",
    "show_price_service": "service_price",
    "show_price_taxcar": "tax_car_price",
    "show_price_fine": "overdue_fine_price",
    "show_price_addon": "price_addon",
    "show_price_payment": "payment_amount",
    "distax": "tax_amount",
    "show_ems_price": "ems_amount",
    "show_discount_ins": "ins_discount",
    "discount_mkt": "mkt_discount",
    "discount_government": "goverment_discount",
    "discount_government_fin": "fin_goverment_discount",
    "discount_government_ins": "ins_goverment_discount",
    "coupon_addon": "discount_addon",
    "worksend": "work_type",
    "chanel": "contact_channel",
    "chanel_key": "key_channel"
}, inplace=True)
df_merged1

In [ ]:
import numpy as np

df_merged1 = df_merged1.replace(r'^\s*$', np.nan, regex=True)

df_merged1 = df_merged1.where(pd.notnull(df_merged1), None)
df_merged1

In [ ]:
import numpy as np

# แปลงทุกคอลัมน์ใน df_merged1 ถ้ามีค่าเป็น string "NAN" ให้เปลี่ยนเป็น np.nan
df_merged1 = df_merged1.replace("NaN", np.nan)
df_merged1

In [ ]:
df_merged1 = df_merged1.drop_duplicates(subset=['quotation_num'], keep='first')
df_merged1

In [ ]:
df_merged1['installment_number'] = df_merged1['installment_number'].replace({
    '0': '1',
    '03': '3',
    '06': '6',
    '08': '8'
})
df_merged1

In [ ]:
import pandas as pd
import numpy as np

# สมมติ df_merged1 มีคอลัมน์ status_fssp, status_fsp, status_fo
# df_merged1 = pd.DataFrame(...)

def map_status(row):
    # ถ้ามี status_fo ให้ใช้ก่อน
    if pd.notnull(row['status_fo']):
        if row['status_fo'] == '88':
            return 'cancel'
        else:
            return row['status_fo']
    # ถ้าไม่มี status_fo ให้ดูคู่ status_fssp, status_fsp
    s1 = row.get('status_fssp')
    s2 = row.get('status_fsp')
    
    if pd.isnull(s1): s1 = ''
    if pd.isnull(s2): s2 = ''
    
    key = (s1.strip(), s2.strip())
    
    mapping = {
        ('wait', ''): '1',
        ('wait-key', ''): '1',
        ('sendpay', 'sendpay'): '2',
        ('sendpay', 'verify-wait'): '2',
        ('tran-succ', 'sendpay'): '2',
        ('tran-succ', 'verify-wait'): '2',
        ('cancel', '88'): 'cancel',
        ('delete', ''): 'delete',
        ('wait', 'sendpay'): '2',
        ('delete', 'sendpay'): 'delete',
        ('delete', 'wait'): 'delete',
        ('delete', 'wait-key'): 'delete',
        ('wait', 'wait'): '1',
        ('wait', 'wait-key'): '1',
        ('', 'wait'): '1',
        ('tran-succ', 'sendpay'): '2',
        ('cancel', 'cancel'): 'cancel',
        ('delete', 'delete'): 'delete',
        ('active', 'verify'): '6',
        ('active', 'success'): '8',
        ('active', ''): '8',
        ('cancel', ''): 'cancel'
    }
    
    # ถ้าไม่ match ให้เป็น None หรือค่า default ที่ต้องการ
    return mapping.get(key, None)

# เพิ่มคอลัมน์ใหม่
df_merged1['status'] = df_merged1.apply(map_status, axis=1)


In [ ]:
df_merged1 = df_merged1[['quotation_num', 'status_fssp', 'status_fsp', 'status_fo', 'status']]
df_merged1

In [ ]:
df_merged1.drop(columns=['status_fssp', 'status_fsp', 'status_fo'], inplace=True)
df_merged1

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import numpy as np

# โหลดตัวแปรจาก .env
load_dotenv()

# ดึงค่าจาก environment
user = os.getenv('DB_USER_test')
password = os.getenv('DB_PASSWORD_test')
host = os.getenv('DB_HOST_test')
port = os.getenv('DB_PORT_test')  
database = 'fininsurance'

# สร้าง engine สำหรับเชื่อมต่อฐานข้อมูล
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# SQL query
query = """
SELECT *
FROM fact_sales_quotation 
"""

df6 = pd.read_sql(query, engine)
df6

In [ ]:
df6 = df6.drop(columns=['create_at', 'update_at', 'status'])
df6

In [ ]:
df_result1 = pd.merge(df_merged1, df6, on=['quotation_num'], how='right')
df_result1

In [ ]:
df_result1 = df_result1.drop_duplicates(subset=['quotation_num'], keep='last')
df_result1

In [ ]:
df_result1 = df_result1[['quotation_num', 'status']]
df_result1

In [ ]:
import numpy as np
import pandas as pd

# แก้ NaT, NaN ทั้ง dataframe ให้เป็น None
df_result1 = df_result1.where(pd.notnull(df_result1), None)
df_result1

In [ ]:
import os
from sqlalchemy import create_engine, MetaData, Table, update
from sqlalchemy import text

user = os.getenv('DB_USER_test')
password = os.getenv('DB_PASSWORD_test')
host = os.getenv('DB_HOST_test')
port = os.getenv('DB_PORT_test')
database = 'fininsurance'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

metadata = MetaData()
table = Table('fact_sales_quotation', metadata, autoload_with=engine)

records = df_result1.to_dict(orient='records')

chunk_size = 5000

for start in range(0, len(records), chunk_size):
    end = start + chunk_size
    chunk = records[start:end]

    print(f"🔄 Updating chunk {start // chunk_size + 1}: records {start} to {end - 1}")

    with engine.begin() as conn:
        for record in chunk:
            # ตรวจสอบว่ามี quotation_num และ status หรือไม่
            if 'quotation_num' not in record or pd.isna(record['quotation_num']):
                print(f"⚠️ Skip row: no quotation_num: {record}")
                continue
            if 'status' not in record or pd.isna(record['status']):
                print(f"⚠️ Skip row: no status: {record}")
                continue

            # ✅ Update เฉพาะคอลัมน์เดียว
            stmt = (
                update(table)
                .where(table.c.quotation_num == record['quotation_num'])
                .values(status=record['status'])
            ) 
            conn.execute(stmt)

print("✅ Update status completed successfully.")

In [ ]:
# columns_to_save = ['status_fssp', 'status_fsp', 'status_fo']
# df_selected = df_merged1[columns_to_save]
# df_selected.to_excel('status.xlsx', index=False, engine='openpyxl')

In [ ]:
# df_merged2.to_excel('quo.xlsx', index=False, engine='openpyxl')